### Decision tree Estimator on merged_emp data 

- This Python notebook demonstrates creating an ML Pipeline to preprocess a dataset, train a Machine Learning model, and make predictions.

- Data: The dataset contains    merged_emp data  

- Goal: We want to learn to predict  deaths  from features such as patient data (comprising of Adult_smoking_percentage,	Adult_obesity_percentage and Excessive_drinking_percentage), various healthcare data like hospitals, physician, nurses, surgean specialities, etc., socio econonmic factors like life rank, quality of life, social-ecomonica factors rank, physical environment rank, etc. and other features like popluation, etc.  

- Approach: We will use Spark ML Pipelines, which help users piece together parts of a workflow such as feature processing and model training. We will also demonstrate model selection (a.k.a. hyperparameter tuning) using Cross Validation in order to fine-tune and improve our ML model.

In [2]:
%sh 
pip install mleap

Requirement already satisfied: mleap in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (0.8.1)
Requirement already satisfied: argparse>=1.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.4.0)
Requirement already satisfied: nose-exclude>=0.5.0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (0.5.0)
Requirement already satisfied: scipy>=0.13.0b1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.2.1)
Requirement already satisfied: numpy>=1.8.2 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.18.5)
Requirement already satisfied: six>=1.10.0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.12.0)
Requirement already satisfied: pandas>=0.18.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.0.4)
Requirement already satisfied: scikit-learn>=0.18.dev0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (0.20.3)
Requirement already satisfied: nose in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from nose-exclude>=0.5.0->mleap) (1.3.7)
Requirement already satisfied: python-dateutil>=2.6.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from pandas>=0.18.1->mleap) (2.8.0)
Requirement already satisfied: pytz>=2017.2 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from pandas>=0.18.1->mleap) (2018.9)

In [3]:
from pyspark.sql.functions import log

In [4]:
# read data from the output of exploratory data analysis which is in parquet file format
df = spark.read.parquet('/databricks/driver/spark_merged_emp_modeling1.parquet')

In [5]:
display(df)

county,cases,deaths,POP_ESTIMATE_2018,All_Specialties_AAMC,Total_nurse_practitioners_2019,Total_physician_assistants_2019,Total_Hospitals_2019,Total_Primary_Care_Physicians_2019,Surgery_specialists_2019,Emergency_Medicine_specialists_2019,Total_Specialist_Physicians_2019,ICU_Beds,pop_fraction,Length_of_Life_rank,Quality_of_Life_rank,Health_Behaviors_rank,Clinical_Care_rank,Social_Economic_Factors_rank,Physical_Environment_rank,Adult_smoking_percentage,Adult_obesity_percentage,Excessive_drinking_percentage,Population_per_sq_mile,House_per_sq_mile,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
Lee,7.117022517792556,0.5833625014584063,17142.0,2.2945925621281065,0.4171807667133357,0.20632844143040488,0.013571651557577879,1.257836399486641,0.1581392439621981,0.14319075819624313,1.2690477639715319,0.0,0.001685835,41.0,46.0,44.0,41.0,42.0,41.0,21.0,40.0,13.0,46.9,19.0,266.0,4.0,33199.0,63.29767965
Lexington,1.667615716261287,0.08473657094823613,295032.0,2.2945925621627485,0.4171807668320725,0.20632844145719786,0.013571651532715095,1.2578363994414166,0.158139243980314,0.14319075822283683,1.2690477636324196,0.18642045608611948,0.029015011,9.0,10.0,11.0,6.0,3.0,7.0,16.0,32.0,19.0,375.4,163.0,3510.0,2.3,60627.0,115.5922897
Marion,1.5464415735043011,0.12887013112535842,31039.0,2.294592562260382,0.41718076677728017,0.20632844144463416,0.013571651535165435,1.2578363993685362,0.15813924395115822,0.14319075820741647,1.2690477637810496,0.32217532781339603,0.00305254,40.0,45.0,41.0,44.0,43.0,19.0,20.0,44.0,14.0,67.6,30.6,564.0,4.3,34365.0,65.52079163
Marlboro,2.272899462080461,0.0,26398.0,2.294592562315327,0.41718076672475185,0.20632844143495718,0.013571651526630805,1.2578363993484354,0.15813924395787562,0.1431907582013789,1.2690477638457458,0.0,0.002596119,43.0,39.0,40.0,39.0,45.0,44.0,21.0,39.0,15.0,60.3,25.2,444.0,4.8,33534.0,63.93639536
McCormick,0.7438894792773645,0.10626992561105207,9410.0,2.2945925621679066,0.41718076673751325,0.206328441445271,0.013571651540913922,1.2578363995749204,0.15813924399574922,0.14319075823591926,1.2690477640807651,0.0,9.25429E-4,46.0,25.0,9.0,4.0,34.0,13.0,17.0,34.0,14.0,28.5,15.2,104.0,3.1,44683.0,85.19323533
Newberry,0.8047767393561787,0.02596053997923157,38520.0,2.294592562305296,0.41718076661474557,0.2063284414330218,0.013571651531671856,1.2578363995327102,0.1581392439771547,0.14319075822949118,1.269047763759086,0.181723779854621,0.003788261,10.0,21.0,23.0,21.0,15.0,16.0,18.0,36.0,16.0,59.5,28.4,514.0,2.7,48029.0,91.57276592
Oconee,0.49761400464439737,0.0,78374.0,2.2945925625845303,0.41718076670834714,0.2063284414474188,0.01357165152984408,1.257836399443693,0.15813924400949295,0.14319075815959373,1.2690477637992192,0.15311200142904535,0.007707715,16.0,4.0,14.0,12.0,12.0,15.0,17.0,32.0,16.0,118.6,61.9,971.0,2.8,50529.0,96.33930104
Orangeburg,1.2193158027929234,0.023005958543262706,86934.0,2.294592562173603,0.41718076667356846,0.20632844146133847,0.013571651528745943,1.2578363988773091,0.15813924390917247,0.14319075816136378,1.2690477638208295,0.23005958543262706,0.00854955,34.0,38.0,27.0,26.0,38.0,37.0,17.0,41.0,13.0,83.6,38.4,1655.0,4.8,37134.0,70.80020591
Pickens,0.6803428928179802,0.008004034033152708,124937.0,2.294592562651576,0.41718076670642007,0.20632844145449308,0.013571651536374333,1.257836399145169,0.1581392439389452,0.14319075822214394,1.2690477640730926,0.12006051049729063,0.012286967,12.0,8.0,6.0,7.0,9.0,33.0,17.0,32.0,18.0,240.2,103.2,1604.0,2.8,48794.0,93.03132567
Richland,2.814924163482691,0.142314075103238,414576.0,2.294592562280499,0.4171807666145652,0.20632844144378837,0.013571651533132646,1.2578363993574158,0.15813924397456677,0.1431907582204469,1.2690477637393385,0.5089537262166648,0.040771601,11.0,14.0,17.0,3.0,21.0,27.0,16.0,33.0,19.0,507.9,213.6,5579.0,2.8,52611.0,100.3088715


In [6]:
df_temp = df
from pyspark.sql.functions import col

#renaming column deaths in order to rearrange it to the end of the table.
df = df.withColumnRenamed('deaths', 'zlabel')

In [7]:
#sort columns according to column name
df = df.select(sorted(df.columns))

In [8]:
df.count()

Out[133]: 2920

In [9]:
df.dtypes

Out[134]: [('Adult_obesity_percentage', 'double'),
 ('Adult_smoking_percentage', 'double'),
 ('All_Specialties_AAMC', 'double'),
 ('Clinical_Care_rank', 'double'),
 ('Emergency_Medicine_specialists_2019', 'double'),
 ('Excessive_drinking_percentage', 'double'),
 ('Health_Behaviors_rank', 'double'),
 ('House_per_sq_mile', 'double'),
 ('ICU_Beds', 'double'),
 ('Length_of_Life_rank', 'double'),
 ('Med_HH_Income_Percent_of_State_Total_2018', 'double'),
 ('Median_Household_Income_2018', 'double'),
 ('POP_ESTIMATE_2018', 'double'),
 ('Physical_Environment_rank', 'double'),
 ('Population_per_sq_mile', 'double'),
 ('Quality_of_Life_rank', 'double'),
 ('Social_Economic_Factors_rank', 'double'),
 ('Surgery_specialists_2019', 'double'),
 ('Total_Hospitals_2019', 'double'),
 ('Total_Primary_Care_Physicians_2019', 'double'),
 ('Total_Specialist_Physicians_2019', 'double'),
 ('Total_nurse_practitioners_2019', 'double'),
 ('Total_physician_assistants_2019', 'double'),
 ('Unemployed_2019', 'double'),
 ('Unemployment_rate_2019', 'double'),
 ('cases', 'double'),
 ('county', 'string'),
 ('pop_fraction', 'double'),
 ('zlabel', 'double')]

In [10]:
display(df)

Adult_obesity_percentage,Adult_smoking_percentage,All_Specialties_AAMC,Clinical_Care_rank,Emergency_Medicine_specialists_2019,Excessive_drinking_percentage,Health_Behaviors_rank,House_per_sq_mile,ICU_Beds,Length_of_Life_rank,Med_HH_Income_Percent_of_State_Total_2018,Median_Household_Income_2018,POP_ESTIMATE_2018,Physical_Environment_rank,Population_per_sq_mile,Quality_of_Life_rank,Social_Economic_Factors_rank,Surgery_specialists_2019,Total_Hospitals_2019,Total_Primary_Care_Physicians_2019,Total_Specialist_Physicians_2019,Total_nurse_practitioners_2019,Total_physician_assistants_2019,Unemployed_2019,Unemployment_rate_2019,cases,county,pop_fraction,zlabel
40.0,21.0,2.2945925621281065,41.0,0.14319075819624313,13.0,44.0,19.0,0.0,41.0,63.29767965,33199.0,17142.0,41.0,46.9,46.0,42.0,0.1581392439621981,0.013571651557577879,1.257836399486641,1.2690477639715319,0.4171807667133357,0.20632844143040488,266.0,4.0,7.117022517792556,Lee,0.001685835,0.5833625014584063
32.0,16.0,2.2945925621627485,6.0,0.14319075822283683,19.0,11.0,163.0,0.18642045608611948,9.0,115.5922897,60627.0,295032.0,7.0,375.4,10.0,3.0,0.158139243980314,0.013571651532715095,1.2578363994414166,1.2690477636324196,0.4171807668320725,0.20632844145719786,3510.0,2.3,1.667615716261287,Lexington,0.029015011,0.08473657094823613
44.0,20.0,2.294592562260382,44.0,0.14319075820741647,14.0,41.0,30.6,0.32217532781339603,40.0,65.52079163,34365.0,31039.0,19.0,67.6,45.0,43.0,0.15813924395115822,0.013571651535165435,1.2578363993685362,1.2690477637810496,0.41718076677728017,0.20632844144463416,564.0,4.3,1.5464415735043011,Marion,0.00305254,0.12887013112535842
39.0,21.0,2.294592562315327,39.0,0.1431907582013789,15.0,40.0,25.2,0.0,43.0,63.93639536,33534.0,26398.0,44.0,60.3,39.0,45.0,0.15813924395787562,0.013571651526630805,1.2578363993484354,1.2690477638457458,0.41718076672475185,0.20632844143495718,444.0,4.8,2.272899462080461,Marlboro,0.002596119,0.0
34.0,17.0,2.2945925621679066,4.0,0.14319075823591926,14.0,9.0,15.2,0.0,46.0,85.19323533,44683.0,9410.0,13.0,28.5,25.0,34.0,0.15813924399574922,0.013571651540913922,1.2578363995749204,1.2690477640807651,0.41718076673751325,0.206328441445271,104.0,3.1,0.7438894792773645,McCormick,9.25429E-4,0.10626992561105207
36.0,18.0,2.294592562305296,21.0,0.14319075822949118,16.0,23.0,28.4,0.181723779854621,10.0,91.57276592,48029.0,38520.0,16.0,59.5,21.0,15.0,0.1581392439771547,0.013571651531671856,1.2578363995327102,1.269047763759086,0.41718076661474557,0.2063284414330218,514.0,2.7,0.8047767393561787,Newberry,0.003788261,0.02596053997923157
32.0,17.0,2.2945925625845303,12.0,0.14319075815959373,16.0,14.0,61.9,0.15311200142904535,16.0,96.33930104,50529.0,78374.0,15.0,118.6,4.0,12.0,0.15813924400949295,0.01357165152984408,1.257836399443693,1.2690477637992192,0.41718076670834714,0.2063284414474188,971.0,2.8,0.49761400464439737,Oconee,0.007707715,0.0
41.0,17.0,2.294592562173603,26.0,0.14319075816136378,13.0,27.0,38.4,0.23005958543262706,34.0,70.80020591,37134.0,86934.0,37.0,83.6,38.0,38.0,0.15813924390917247,0.013571651528745943,1.2578363988773091,1.2690477638208295,0.41718076667356846,0.20632844146133847,1655.0,4.8,1.2193158027929234,Orangeburg,0.00854955,0.023005958543262706
32.0,17.0,2.294592562651576,7.0,0.14319075822214394,18.0,6.0,103.2,0.12006051049729063,12.0,93.03132567,48794.0,124937.0,33.0,240.2,8.0,9.0,0.1581392439389452,0.013571651536374333,1.257836399145169,1.2690477640730926,0.41718076670642007,0.20632844145449308,1604.0,2.8,0.6803428928179802,Pickens,0.012286967,0.008004034033152708
33.0,16.0,2.294592562280499,3.0,0.1431907582204469,19.0,17.0,213.6,0.5089537262166648,11.0,100.3088715,52611.0,414576.0,27.0,507.9,14.0,21.0,0.15813924397456677,0.013571651533132646,1.2578363993574158,1.2690477637393385,0.4171807666145652,0.20632844144378837,5579.0,2.8,2.814924163482691,Richland,0.040771601,0.142314075103238


In [11]:
#rename column back to label
df = df.withColumnRenamed('zlabel', 'label')

In [12]:
display(df)

Adult_obesity_percentage,Adult_smoking_percentage,All_Specialties_AAMC,Clinical_Care_rank,Emergency_Medicine_specialists_2019,Excessive_drinking_percentage,Health_Behaviors_rank,House_per_sq_mile,ICU_Beds,Length_of_Life_rank,Med_HH_Income_Percent_of_State_Total_2018,Median_Household_Income_2018,POP_ESTIMATE_2018,Physical_Environment_rank,Population_per_sq_mile,Quality_of_Life_rank,Social_Economic_Factors_rank,Surgery_specialists_2019,Total_Hospitals_2019,Total_Primary_Care_Physicians_2019,Total_Specialist_Physicians_2019,Total_nurse_practitioners_2019,Total_physician_assistants_2019,Unemployed_2019,Unemployment_rate_2019,cases,county,pop_fraction,label
40.0,21.0,2.2945925621281065,41.0,0.14319075819624313,13.0,44.0,19.0,0.0,41.0,63.29767965,33199.0,17142.0,41.0,46.9,46.0,42.0,0.1581392439621981,0.013571651557577879,1.257836399486641,1.2690477639715319,0.4171807667133357,0.20632844143040488,266.0,4.0,7.117022517792556,Lee,0.001685835,0.5833625014584063
32.0,16.0,2.2945925621627485,6.0,0.14319075822283683,19.0,11.0,163.0,0.18642045608611948,9.0,115.5922897,60627.0,295032.0,7.0,375.4,10.0,3.0,0.158139243980314,0.013571651532715095,1.2578363994414166,1.2690477636324196,0.4171807668320725,0.20632844145719786,3510.0,2.3,1.667615716261287,Lexington,0.029015011,0.08473657094823613
44.0,20.0,2.294592562260382,44.0,0.14319075820741647,14.0,41.0,30.6,0.32217532781339603,40.0,65.52079163,34365.0,31039.0,19.0,67.6,45.0,43.0,0.15813924395115822,0.013571651535165435,1.2578363993685362,1.2690477637810496,0.41718076677728017,0.20632844144463416,564.0,4.3,1.5464415735043011,Marion,0.00305254,0.12887013112535842
39.0,21.0,2.294592562315327,39.0,0.1431907582013789,15.0,40.0,25.2,0.0,43.0,63.93639536,33534.0,26398.0,44.0,60.3,39.0,45.0,0.15813924395787562,0.013571651526630805,1.2578363993484354,1.2690477638457458,0.41718076672475185,0.20632844143495718,444.0,4.8,2.272899462080461,Marlboro,0.002596119,0.0
34.0,17.0,2.2945925621679066,4.0,0.14319075823591926,14.0,9.0,15.2,0.0,46.0,85.19323533,44683.0,9410.0,13.0,28.5,25.0,34.0,0.15813924399574922,0.013571651540913922,1.2578363995749204,1.2690477640807651,0.41718076673751325,0.206328441445271,104.0,3.1,0.7438894792773645,McCormick,9.25429E-4,0.10626992561105207
36.0,18.0,2.294592562305296,21.0,0.14319075822949118,16.0,23.0,28.4,0.181723779854621,10.0,91.57276592,48029.0,38520.0,16.0,59.5,21.0,15.0,0.1581392439771547,0.013571651531671856,1.2578363995327102,1.269047763759086,0.41718076661474557,0.2063284414330218,514.0,2.7,0.8047767393561787,Newberry,0.003788261,0.02596053997923157
32.0,17.0,2.2945925625845303,12.0,0.14319075815959373,16.0,14.0,61.9,0.15311200142904535,16.0,96.33930104,50529.0,78374.0,15.0,118.6,4.0,12.0,0.15813924400949295,0.01357165152984408,1.257836399443693,1.2690477637992192,0.41718076670834714,0.2063284414474188,971.0,2.8,0.49761400464439737,Oconee,0.007707715,0.0
41.0,17.0,2.294592562173603,26.0,0.14319075816136378,13.0,27.0,38.4,0.23005958543262706,34.0,70.80020591,37134.0,86934.0,37.0,83.6,38.0,38.0,0.15813924390917247,0.013571651528745943,1.2578363988773091,1.2690477638208295,0.41718076667356846,0.20632844146133847,1655.0,4.8,1.2193158027929234,Orangeburg,0.00854955,0.023005958543262706
32.0,17.0,2.294592562651576,7.0,0.14319075822214394,18.0,6.0,103.2,0.12006051049729063,12.0,93.03132567,48794.0,124937.0,33.0,240.2,8.0,9.0,0.1581392439389452,0.013571651536374333,1.257836399145169,1.2690477640730926,0.41718076670642007,0.20632844145449308,1604.0,2.8,0.6803428928179802,Pickens,0.012286967,0.008004034033152708
33.0,16.0,2.294592562280499,3.0,0.1431907582204469,19.0,17.0,213.6,0.5089537262166648,11.0,100.3088715,52611.0,414576.0,27.0,507.9,14.0,21.0,0.15813924397456677,0.013571651533132646,1.2578363993574158,1.2690477637393385,0.4171807666145652,0.20632844144378837,5579.0,2.8,2.814924163482691,Richland,0.040771601,0.142314075103238


In [13]:
df.where( df['label'].isNull() ).count()

Out[138]: 0

In [14]:
df.where( df['pop_fraction'].isNull() ).count()

Out[139]: 30

In [15]:
#define a function to identify the count of nulls in the columns
def count_nulls(df):
    null_counts = []          #make an empty list to hold our results
    for col in df.dtypes:     #iterate through the column data types we saw above, e.g. ('C0', 'bigint')
        cname = col[0]        #splits out the column name, e.g. 'C0'    
        ctype = col[1]        #splits out the column type, e.g. 'bigint'
        if ctype != 'string': #skip processing string columns for efficiency (can't have nulls)
            nulls = df.where( df[cname].isNull() ).count()
            result = tuple([cname, nulls])  #new tuple, (column name, null count)
            null_counts.append(result)      #put the new tuple in our result list
    return null_counts

null_counts = count_nulls(df)

In [16]:
null_counts

Out[141]: [('Adult_obesity_percentage', 0),
 ('Adult_smoking_percentage', 0),
 ('All_Specialties_AAMC', 0),
 ('Clinical_Care_rank', 0),
 ('Emergency_Medicine_specialists_2019', 0),
 ('Excessive_drinking_percentage', 0),
 ('Health_Behaviors_rank', 0),
 ('House_per_sq_mile', 0),
 ('ICU_Beds', 0),
 ('Length_of_Life_rank', 0),
 ('Med_HH_Income_Percent_of_State_Total_2018', 0),
 ('Median_Household_Income_2018', 0),
 ('POP_ESTIMATE_2018', 30),
 ('Physical_Environment_rank', 0),
 ('Population_per_sq_mile', 0),
 ('Quality_of_Life_rank', 0),
 ('Social_Economic_Factors_rank', 0),
 ('Surgery_specialists_2019', 0),
 ('Total_Hospitals_2019', 0),
 ('Total_Primary_Care_Physicians_2019', 0),
 ('Total_Specialist_Physicians_2019', 0),
 ('Total_nurse_practitioners_2019', 0),
 ('Total_physician_assistants_2019', 0),
 ('Unemployed_2019', 0),
 ('Unemployment_rate_2019', 0),
 ('cases', 0),
 ('pop_fraction', 30),
 ('label', 0)]

In [17]:
display(df)

Adult_obesity_percentage,Adult_smoking_percentage,All_Specialties_AAMC,Clinical_Care_rank,Emergency_Medicine_specialists_2019,Excessive_drinking_percentage,Health_Behaviors_rank,House_per_sq_mile,ICU_Beds,Length_of_Life_rank,Med_HH_Income_Percent_of_State_Total_2018,Median_Household_Income_2018,POP_ESTIMATE_2018,Physical_Environment_rank,Population_per_sq_mile,Quality_of_Life_rank,Social_Economic_Factors_rank,Surgery_specialists_2019,Total_Hospitals_2019,Total_Primary_Care_Physicians_2019,Total_Specialist_Physicians_2019,Total_nurse_practitioners_2019,Total_physician_assistants_2019,Unemployed_2019,Unemployment_rate_2019,cases,county,pop_fraction,label
40.0,21.0,2.2945925621281065,41.0,0.14319075819624313,13.0,44.0,19.0,0.0,41.0,63.29767965,33199.0,17142.0,41.0,46.9,46.0,42.0,0.1581392439621981,0.013571651557577879,1.257836399486641,1.2690477639715319,0.4171807667133357,0.20632844143040488,266.0,4.0,7.117022517792556,Lee,0.001685835,0.5833625014584063
32.0,16.0,2.2945925621627485,6.0,0.14319075822283683,19.0,11.0,163.0,0.18642045608611948,9.0,115.5922897,60627.0,295032.0,7.0,375.4,10.0,3.0,0.158139243980314,0.013571651532715095,1.2578363994414166,1.2690477636324196,0.4171807668320725,0.20632844145719786,3510.0,2.3,1.667615716261287,Lexington,0.029015011,0.08473657094823613
44.0,20.0,2.294592562260382,44.0,0.14319075820741647,14.0,41.0,30.6,0.32217532781339603,40.0,65.52079163,34365.0,31039.0,19.0,67.6,45.0,43.0,0.15813924395115822,0.013571651535165435,1.2578363993685362,1.2690477637810496,0.41718076677728017,0.20632844144463416,564.0,4.3,1.5464415735043011,Marion,0.00305254,0.12887013112535842
39.0,21.0,2.294592562315327,39.0,0.1431907582013789,15.0,40.0,25.2,0.0,43.0,63.93639536,33534.0,26398.0,44.0,60.3,39.0,45.0,0.15813924395787562,0.013571651526630805,1.2578363993484354,1.2690477638457458,0.41718076672475185,0.20632844143495718,444.0,4.8,2.272899462080461,Marlboro,0.002596119,0.0
34.0,17.0,2.2945925621679066,4.0,0.14319075823591926,14.0,9.0,15.2,0.0,46.0,85.19323533,44683.0,9410.0,13.0,28.5,25.0,34.0,0.15813924399574922,0.013571651540913922,1.2578363995749204,1.2690477640807651,0.41718076673751325,0.206328441445271,104.0,3.1,0.7438894792773645,McCormick,9.25429E-4,0.10626992561105207
36.0,18.0,2.294592562305296,21.0,0.14319075822949118,16.0,23.0,28.4,0.181723779854621,10.0,91.57276592,48029.0,38520.0,16.0,59.5,21.0,15.0,0.1581392439771547,0.013571651531671856,1.2578363995327102,1.269047763759086,0.41718076661474557,0.2063284414330218,514.0,2.7,0.8047767393561787,Newberry,0.003788261,0.02596053997923157
32.0,17.0,2.2945925625845303,12.0,0.14319075815959373,16.0,14.0,61.9,0.15311200142904535,16.0,96.33930104,50529.0,78374.0,15.0,118.6,4.0,12.0,0.15813924400949295,0.01357165152984408,1.257836399443693,1.2690477637992192,0.41718076670834714,0.2063284414474188,971.0,2.8,0.49761400464439737,Oconee,0.007707715,0.0
41.0,17.0,2.294592562173603,26.0,0.14319075816136378,13.0,27.0,38.4,0.23005958543262706,34.0,70.80020591,37134.0,86934.0,37.0,83.6,38.0,38.0,0.15813924390917247,0.013571651528745943,1.2578363988773091,1.2690477638208295,0.41718076667356846,0.20632844146133847,1655.0,4.8,1.2193158027929234,Orangeburg,0.00854955,0.023005958543262706
32.0,17.0,2.294592562651576,7.0,0.14319075822214394,18.0,6.0,103.2,0.12006051049729063,12.0,93.03132567,48794.0,124937.0,33.0,240.2,8.0,9.0,0.1581392439389452,0.013571651536374333,1.257836399145169,1.2690477640730926,0.41718076670642007,0.20632844145449308,1604.0,2.8,0.6803428928179802,Pickens,0.012286967,0.008004034033152708
33.0,16.0,2.294592562280499,3.0,0.1431907582204469,19.0,17.0,213.6,0.5089537262166648,11.0,100.3088715,52611.0,414576.0,27.0,507.9,14.0,21.0,0.15813924397456677,0.013571651533132646,1.2578363993574158,1.2690477637393385,0.4171807666145652,0.20632844144378837,5579.0,2.8,2.814924163482691,Richland,0.040771601,0.142314075103238


In [18]:

df = df.drop("pop_fraction")
df_temp=df_temp.drop("pop_fraction")

In [19]:
df_described = df.describe()
df_described.show()

+-------+------------------------+------------------------+--------------------+------------------+-----------------------------------+-----------------------------+---------------------+------------------+------------------+-------------------+-----------------------------------------+----------------------------+------------------+-------------------------+----------------------+--------------------+----------------------------+------------------------+--------------------+----------------------------------+--------------------------------+------------------------------+-------------------------------+------------------+----------------------+--------------------+---------+-------------------+
summary|Adult_obesity_percentage|Adult_smoking_percentage|All_Specialties_AAMC|Clinical_Care_rank|Emergency_Medicine_specialists_2019|Excessive_drinking_percentage|Health_Behaviors_rank| House_per_sq_mile| ICU_Beds|Length_of_Life_rank|Med_HH_Income_Percent_of_State_Total_2018|Median_Household_Income_2018| POP_ESTIMATE_2018|Physical_Environment_rank|Population_per_sq_mile|Quality_of_Life_rank|Social_Economic_Factors_rank|Surgery_specialists_2019|Total_Hospitals_2019|Total_Primary_Care_Physicians_2019|Total_Specialist_Physicians_2019|Total_nurse_practitioners_2019|Total_physician_assistants_2019| Unemployed_2019|Unemployment_rate_2019| cases| county| label|
+-------+------------------------+------------------------+--------------------+------------------+-----------------------------------+-----------------------------+---------------------+------------------+------------------+-------------------+-----------------------------------------+----------------------------+------------------+-------------------------+----------------------+--------------------+----------------------------+------------------------+--------------------+----------------------------------+--------------------------------+------------------------------+-------------------------------+------------------+----------------------+--------------------+---------+-------------------+
 count| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2890| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920|
 mean| 33.068166089965395| 17.554671280276814| 2.5584090761475906| 46.80800000000001| 0.15517237341472012| 17.51280276816609| 48.40382608695652| 97.04292142609899|0.1416556103918164| 46.90747826086957| 90.08975735053288| 53112.23287197231|109859.56712802769| 49.47965217391304| 223.1832467982001| 47.572173913043486| 47.64799999999999| 0.15678187709488492|0.022360032084671223| 1.3832442105763176| 1.4630687143103664| 0.5153777259287423| 0.26318240467730636|2007.0051903114188| 4.006297577854671| 2.3734327601716774| null|0.09867339905928069|
 stddev| 5.429846945234563| 3.5345823419759763| 0.4277294587437189| 40.29271118768756| 0.05169390510679405| 3.175252580101332| 41.677558663716596|346.20795974896356|0.5429733388072986| 41.3505116416244| 20.25639569033717| 14034.702653941114| 338843.6418601639| 42.14004433599951| 776.2830201250127| 41.14424435336909| 41.19115348867994| 0.03422826532246513|0.012286295306214305| 0.26311556941990233| 0.3433993192893988| 0.11954393548085461| 0.10668379320578586|6631.4655606166825| 1.4121948910978879| 4.992203702745789| null| 0.2214383392342358|
 min| 12.0| 6.0| 1.9132571908978813| 1.0| 0.06574212065070041| 8.0| 1.0| 0.0| 0.0| 1.0| 43.86023433| 25385.0| 655.0| 1.0| 0.0| 1.0| 1.0| 0.08493861579414375|0.007237603276393...| 0.9217835054678007| 0.7952306679389312| 0.27715601667113704| 0.035827532226287644| 9.0| 1.4|0.015645534764378247|Abbeville| 0.0|
 max| 58.0| 41.0| 8.470293470756134| 244.0| 0.5324184467332427| 29.0| 244.0| 8041.8|27.452256944444443| 244.0| 263.5734627| 140382.0| 1.0105518E7| 244.0| 17179.2| 242.0| 244.0| 0.6050209621968667| 0.06460863624039766| 4.52128606102882| 5.936323323202198| 0.9075434334231974| 0.46919198642294396| 227288.0| 18.3| 125.40864511442064| varro| 3.073320649787

In [20]:
from pyspark.sql.functions import skewness, kurtosis
from pyspark.sql.functions import var_pop, var_samp, stddev, stddev_pop, sumDistinct, ntile
df.select(skewness('label')).show()


+-----------------+
 skewness(label)|
+-----------------+
5.052373505505389|
+-----------------+

In [21]:
from pyspark.sql import Row

columns = df_described.columns  #list of column names
funcs   = [skewness, kurtosis]  #list of functions we want to include (imported earlier)
fnames  = ['skew', 'kurtosis']  #a list of strings describing the functions in the same order

def new_item(func, column):
    """
    This function takes in an aggregation function and a column name, then applies the aggregation to the
    column, collects it and returns a value.  The value is in string format despite being a number, 
    because that matches the output of describe.
    """
    return str(df.select(func(column)).collect()[0][0])

new_data = []
for func, fname in zip(funcs, fnames):
    row_dict = {'summary':fname}  #each row object begins with an entry for "summary"
    for column in columns[1:]:
        row_dict[column] = new_item(func, column)
    new_data.append(Row(**row_dict))  #using ** tells Python to unpack the entries of the dictionary
    
print(new_data)

[Row(Adult_obesity_percentage='-0.1008159824354525', Adult_smoking_percentage='0.7888538623773389', All_Specialties_AAMC='2.0576954363038213', Clinical_Care_rank='1.7058940335167307', Emergency_Medicine_specialists_2019='1.5444555671512559', Excessive_drinking_percentage='0.1626094363997207', Health_Behaviors_rank='1.7253614915641238', House_per_sq_mile='11.898085679978479', ICU_Beds='43.758898615649', Length_of_Life_rank='1.8024353372638178', Med_HH_Income_Percent_of_State_Total_2018='1.2986251553117345', Median_Household_Income_2018='1.493178410725302', POP_ESTIMATE_2018='13.737687210118974', Physical_Environment_rank='1.7191036679887948', Population_per_sq_mile='11.12358476244174', Quality_of_Life_rank='1.73622169229188', Social_Economic_Factors_rank='1.7502756489470626', Surgery_specialists_2019='1.331876470131877', Total_Hospitals_2019='1.4288609483685522', Total_Primary_Care_Physicians_2019='1.4162082709479569', Total_Specialist_Physicians_2019='2.0449781498717825', Total_nurse_practitioners_2019='0.7515580115015815', Total_physician_assistants_2019='-0.026136439040272304', Unemployed_2019='17.548934220386318', Unemployment_rate_2019='2.2345545614658917', cases='9.77669676790574', county='None', label='5.052373505505389', summary='skew'), Row(Adult_obesity_percentage='0.5235663388625418', Adult_smoking_percentage='2.423224982545177', All_Specialties_AAMC='14.468154845472782', Clinical_Care_rank='4.017975712907111', Emergency_Medicine_specialists_2019='3.561727916341921', Excessive_drinking_percentage='0.014320400789793908', Health_Behaviors_rank='4.019714534274933', House_per_sq_mile='191.96151317133055', ICU_Beds='2190.8986988929532', Length_of_Life_rank='4.37005593755238', Med_HH_Income_Percent_of_State_Total_2018='4.33035562176173', Median_Household_Income_2018='3.9156612073332377', POP_ESTIMATE_2018='310.56575386995496', Physical_Environment_rank='3.931636227904445', Population_per_sq_mile='171.1385861529977', Quality_of_Life_rank='4.0794287584919084', Social_Economic_Factors_rank='4.18519170141111', Surgery_specialists_2019='10.015850138080602', Total_Hospitals_2019='1.5986510181956497', Total_Primary_Care_Physicians_2019='7.217673531343301', Total_Specialist_Physicians_2019='11.971662380288494', Total_nurse_practitioners_2019='1.2392835500043384', Total_physician_assistants_2019='-0.661695938012234', Unemployed_2019='497.7956686450704', Unemployment_rate_2019='11.328895935631742', cases='167.3470216050706', county='None', label='38.98841964641258', summary='kurtosis')]

In [22]:
df_described.collect()

Out[147]: [Row(summary='count', Adult_obesity_percentage='2920', Adult_smoking_percentage='2920', All_Specialties_AAMC='2920', Clinical_Care_rank='2920', Emergency_Medicine_specialists_2019='2920', Excessive_drinking_percentage='2920', Health_Behaviors_rank='2920', House_per_sq_mile='2920', ICU_Beds='2920', Length_of_Life_rank='2920', Med_HH_Income_Percent_of_State_Total_2018='2920', Median_Household_Income_2018='2920', POP_ESTIMATE_2018='2890', Physical_Environment_rank='2920', Population_per_sq_mile='2920', Quality_of_Life_rank='2920', Social_Economic_Factors_rank='2920', Surgery_specialists_2019='2920', Total_Hospitals_2019='2920', Total_Primary_Care_Physicians_2019='2920', Total_Specialist_Physicians_2019='2920', Total_nurse_practitioners_2019='2920', Total_physician_assistants_2019='2920', Unemployed_2019='2920', Unemployment_rate_2019='2920', cases='2920', county='2920', label='2920'),
 Row(summary='mean', Adult_obesity_percentage='33.068166089965395', Adult_smoking_percentage='17.554671280276814', All_Specialties_AAMC='2.5584090761475906', Clinical_Care_rank='46.80800000000001', Emergency_Medicine_specialists_2019='0.15517237341472012', Excessive_drinking_percentage='17.51280276816609', Health_Behaviors_rank='48.40382608695652', House_per_sq_mile='97.04292142609899', ICU_Beds='0.1416556103918164', Length_of_Life_rank='46.90747826086957', Med_HH_Income_Percent_of_State_Total_2018='90.08975735053288', Median_Household_Income_2018='53112.23287197231', POP_ESTIMATE_2018='109859.56712802769', Physical_Environment_rank='49.47965217391304', Population_per_sq_mile='223.1832467982001', Quality_of_Life_rank='47.572173913043486', Social_Economic_Factors_rank='47.64799999999999', Surgery_specialists_2019='0.15678187709488492', Total_Hospitals_2019='0.022360032084671223', Total_Primary_Care_Physicians_2019='1.3832442105763176', Total_Specialist_Physicians_2019='1.4630687143103664', Total_nurse_practitioners_2019='0.5153777259287423', Total_physician_assistants_2019='0.26318240467730636', Unemployed_2019='2007.0051903114188', Unemployment_rate_2019='4.006297577854671', cases='2.3734327601716774', county=None, label='0.09867339905928069'),
 Row(summary='stddev', Adult_obesity_percentage='5.429846945234563', Adult_smoking_percentage='3.5345823419759763', All_Specialties_AAMC='0.4277294587437189', Clinical_Care_rank='40.29271118768756', Emergency_Medicine_specialists_2019='0.05169390510679405', Excessive_drinking_percentage='3.175252580101332', Health_Behaviors_rank='41.677558663716596', House_per_sq_mile='346.20795974896356', ICU_Beds='0.5429733388072986', Length_of_Life_rank='41.3505116416244', Med_HH_Income_Percent_of_State_Total_2018='20.25639569033717', Median_Household_Income_2018='14034.702653941114', POP_ESTIMATE_2018='338843.6418601639', Physical_Environment_rank='42.14004433599951', Population_per_sq_mile='776.2830201250127', Quality_of_Life_rank='41.14424435336909', Social_Economic_Factors_rank='41.19115348867994', Surgery_specialists_2019='0.03422826532246513', Total_Hospitals_2019='0.012286295306214305', Total_Primary_Care_Physicians_2019='0.26311556941990233', Total_Specialist_Physicians_2019='0.3433993192893988', Total_nurse_practitioners_2019='0.11954393548085461', Total_physician_assistants_2019='0.10668379320578586', Unemployed_2019='6631.4655606166825', Unemployment_rate_2019='1.4121948910978879', cases='4.992203702745789', county=None, label='0.2214383392342358'),
 Row(summary='min', Adult_obesity_percentage='12.0', Adult_smoking_percentage='6.0', All_Specialties_AAMC='1.9132571908978813', Clinical_Care_rank='1.0', Emergency_Medicine_specialists_2019='0.06574212065070041', Excessive_drinking_percentage='8.0', Health_Behaviors_rank='1.0', House_per_sq_mile='0.0', ICU_Beds='0.0', Length_of_Life_rank='1.0', Med_HH_Income_Percent_of_State_Total_2018='43.86023433', Median_Household_Income_2018='25385.0', POP_ESTIMATE_2018='655.0', Physical_Environment_rank='1.0', Population_per_sq_mile='0.0', Quality_of_Life_rank='1.0', S

In [23]:
new_describe = sc.parallelize(new_data).toDF()           #turns the results from our loop into a dataframe
new_describe = new_describe.select(df_described.columns) #forces the columns into the same order

expanded_describe = df_described.unionAll(new_describe)  #merges the new stats with the original describe
expanded_describe.show()


/databricks/spark/python/pyspark/sql/dataframe.py:1505: DeprecationWarning: Deprecated in 2.0, use union instead.
 warnings.warn("Deprecated in 2.0, use union instead.", DeprecationWarning)
+--------+------------------------+------------------------+--------------------+------------------+-----------------------------------+-----------------------------+---------------------+------------------+------------------+-------------------+-----------------------------------------+----------------------------+------------------+-------------------------+----------------------+--------------------+----------------------------+------------------------+--------------------+----------------------------------+--------------------------------+------------------------------+-------------------------------+------------------+----------------------+--------------------+---------+-------------------+
 summary|Adult_obesity_percentage|Adult_smoking_percentage|All_Specialties_AAMC|Clinical_Care_rank|Emergency_Medicine_specialists_2019|Excessive_drinking_percentage|Health_Behaviors_rank| House_per_sq_mile| ICU_Beds|Length_of_Life_rank|Med_HH_Income_Percent_of_State_Total_2018|Median_Household_Income_2018| POP_ESTIMATE_2018|Physical_Environment_rank|Population_per_sq_mile|Quality_of_Life_rank|Social_Economic_Factors_rank|Surgery_specialists_2019|Total_Hospitals_2019|Total_Primary_Care_Physicians_2019|Total_Specialist_Physicians_2019|Total_nurse_practitioners_2019|Total_physician_assistants_2019| Unemployed_2019|Unemployment_rate_2019| cases| county| label|
+--------+------------------------+------------------------+--------------------+------------------+-----------------------------------+-----------------------------+---------------------+------------------+------------------+-------------------+-----------------------------------------+----------------------------+------------------+-------------------------+----------------------+--------------------+----------------------------+------------------------+--------------------+----------------------------------+--------------------------------+------------------------------+-------------------------------+------------------+----------------------+--------------------+---------+-------------------+
 count| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2890| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920| 2920|
 mean| 33.068166089965395| 17.554671280276814| 2.5584090761475906| 46.80800000000001| 0.15517237341472012| 17.51280276816609| 48.40382608695652| 97.04292142609899|0.1416556103918164| 46.90747826086957| 90.08975735053288| 53112.23287197231|109859.56712802769| 49.47965217391304| 223.1832467982001| 47.572173913043486| 47.64799999999999| 0.15678187709488492|0.022360032084671223| 1.3832442105763176| 1.4630687143103664| 0.5153777259287423| 0.26318240467730636|2007.0051903114188| 4.006297577854671| 2.3734327601716774| null|0.09867339905928069|
 stddev| 5.429846945234563| 3.5345823419759763| 0.4277294587437189| 40.29271118768756| 0.05169390510679405| 3.175252580101332| 41.677558663716596|346.20795974896356|0.5429733388072986| 41.3505116416244| 20.25639569033717| 14034.702653941114| 338843.6418601639| 42.14004433599951| 776.2830201250127| 41.14424435336909| 41.19115348867994| 0.03422826532246513|0.012286295306214305| 0.26311556941990233| 0.3433993192893988| 0.11954393548085461| 0.10668379320578586|6631.4655606166825| 1.4121948910978879| 4.992203702745789| null| 0.2214383392342358|
 min| 12.0| 6.0| 1.9132571908978813| 1.0| 0.06574212065070041| 8.0| 1.0| 0.0| 0.0| 1.0| 43.86023433| 25385.0| 655.0| 1.0| 0.0| 1.0| 1.0| 0.08493861579414375|0.007237603276393...| 0.9217835054678007| 0.7952306679389312| 0.27715601667113704| 0.035827532226287644| 9.0| 1.4|0.015645534764378247|Abbeville| 0.0|
 max| 58.0| 41.0| 8.470293470756134| 244.0| 0.5324184467332427| 29.0| 244.0| 8041.8|27.452256944444443| 244.0| 263.5734627| 140382.0| 1.0105518E7| 244.0| 17179.

In [24]:
label = df[['label']].collect()


In [25]:
print(label[:5])


[Row(label=0.5833625014584063), Row(label=0.08473657094823613), Row(label=0.12887013112535842), Row(label=0.0), Row(label=0.10626992561105207)]

In [26]:
df.columns


Out[151]: ['Adult_obesity_percentage',
 'Adult_smoking_percentage',
 'All_Specialties_AAMC',
 'Clinical_Care_rank',
 'Emergency_Medicine_specialists_2019',
 'Excessive_drinking_percentage',
 'Health_Behaviors_rank',
 'House_per_sq_mile',
 'ICU_Beds',
 'Length_of_Life_rank',
 'Med_HH_Income_Percent_of_State_Total_2018',
 'Median_Household_Income_2018',
 'POP_ESTIMATE_2018',
 'Physical_Environment_rank',
 'Population_per_sq_mile',
 'Quality_of_Life_rank',
 'Social_Economic_Factors_rank',
 'Surgery_specialists_2019',
 'Total_Hospitals_2019',
 'Total_Primary_Care_Physicians_2019',
 'Total_Specialist_Physicians_2019',
 'Total_nurse_practitioners_2019',
 'Total_physician_assistants_2019',
 'Unemployed_2019',
 'Unemployment_rate_2019',
 'cases',
 'county',
 'label']

In [27]:
df.dtypes

Out[152]: [('Adult_obesity_percentage', 'double'),
 ('Adult_smoking_percentage', 'double'),
 ('All_Specialties_AAMC', 'double'),
 ('Clinical_Care_rank', 'double'),
 ('Emergency_Medicine_specialists_2019', 'double'),
 ('Excessive_drinking_percentage', 'double'),
 ('Health_Behaviors_rank', 'double'),
 ('House_per_sq_mile', 'double'),
 ('ICU_Beds', 'double'),
 ('Length_of_Life_rank', 'double'),
 ('Med_HH_Income_Percent_of_State_Total_2018', 'double'),
 ('Median_Household_Income_2018', 'double'),
 ('POP_ESTIMATE_2018', 'double'),
 ('Physical_Environment_rank', 'double'),
 ('Population_per_sq_mile', 'double'),
 ('Quality_of_Life_rank', 'double'),
 ('Social_Economic_Factors_rank', 'double'),
 ('Surgery_specialists_2019', 'double'),
 ('Total_Hospitals_2019', 'double'),
 ('Total_Primary_Care_Physicians_2019', 'double'),
 ('Total_Specialist_Physicians_2019', 'double'),
 ('Total_nurse_practitioners_2019', 'double'),
 ('Total_physician_assistants_2019', 'double'),
 ('Unemployed_2019', 'double'),
 ('Unemployment_rate_2019', 'double'),
 ('cases', 'double'),
 ('county', 'string'),
 ('label', 'double')]

In [28]:
#drop county since it is string and is not a numerical feature
df = df.drop("county")
df_temp=df_temp.drop("county")


In [29]:
df = df.dropna()
df_temp=df_temp.dropna()

In [30]:
display(df)

Adult_obesity_percentage,Adult_smoking_percentage,All_Specialties_AAMC,Clinical_Care_rank,Emergency_Medicine_specialists_2019,Excessive_drinking_percentage,Health_Behaviors_rank,House_per_sq_mile,ICU_Beds,Length_of_Life_rank,Med_HH_Income_Percent_of_State_Total_2018,Median_Household_Income_2018,POP_ESTIMATE_2018,Physical_Environment_rank,Population_per_sq_mile,Quality_of_Life_rank,Social_Economic_Factors_rank,Surgery_specialists_2019,Total_Hospitals_2019,Total_Primary_Care_Physicians_2019,Total_Specialist_Physicians_2019,Total_nurse_practitioners_2019,Total_physician_assistants_2019,Unemployed_2019,Unemployment_rate_2019,cases,label
40.0,21.0,2.2945925621281065,41.0,0.14319075819624313,13.0,44.0,19.0,0.0,41.0,63.29767965,33199.0,17142.0,41.0,46.9,46.0,42.0,0.1581392439621981,0.013571651557577879,1.257836399486641,1.2690477639715319,0.4171807667133357,0.20632844143040488,266.0,4.0,7.117022517792556,0.5833625014584063
32.0,16.0,2.2945925621627485,6.0,0.14319075822283683,19.0,11.0,163.0,0.18642045608611948,9.0,115.5922897,60627.0,295032.0,7.0,375.4,10.0,3.0,0.158139243980314,0.013571651532715095,1.2578363994414166,1.2690477636324196,0.4171807668320725,0.20632844145719786,3510.0,2.3,1.667615716261287,0.08473657094823613
44.0,20.0,2.294592562260382,44.0,0.14319075820741647,14.0,41.0,30.6,0.32217532781339603,40.0,65.52079163,34365.0,31039.0,19.0,67.6,45.0,43.0,0.15813924395115822,0.013571651535165435,1.2578363993685362,1.2690477637810496,0.41718076677728017,0.20632844144463416,564.0,4.3,1.5464415735043011,0.12887013112535842
39.0,21.0,2.294592562315327,39.0,0.1431907582013789,15.0,40.0,25.2,0.0,43.0,63.93639536,33534.0,26398.0,44.0,60.3,39.0,45.0,0.15813924395787562,0.013571651526630805,1.2578363993484354,1.2690477638457458,0.41718076672475185,0.20632844143495718,444.0,4.8,2.272899462080461,0.0
34.0,17.0,2.2945925621679066,4.0,0.14319075823591926,14.0,9.0,15.2,0.0,46.0,85.19323533,44683.0,9410.0,13.0,28.5,25.0,34.0,0.15813924399574922,0.013571651540913922,1.2578363995749204,1.2690477640807651,0.41718076673751325,0.206328441445271,104.0,3.1,0.7438894792773645,0.10626992561105207
36.0,18.0,2.294592562305296,21.0,0.14319075822949118,16.0,23.0,28.4,0.181723779854621,10.0,91.57276592,48029.0,38520.0,16.0,59.5,21.0,15.0,0.1581392439771547,0.013571651531671856,1.2578363995327102,1.269047763759086,0.41718076661474557,0.2063284414330218,514.0,2.7,0.8047767393561787,0.02596053997923157
32.0,17.0,2.2945925625845303,12.0,0.14319075815959373,16.0,14.0,61.9,0.15311200142904535,16.0,96.33930104,50529.0,78374.0,15.0,118.6,4.0,12.0,0.15813924400949295,0.01357165152984408,1.257836399443693,1.2690477637992192,0.41718076670834714,0.2063284414474188,971.0,2.8,0.49761400464439737,0.0
41.0,17.0,2.294592562173603,26.0,0.14319075816136378,13.0,27.0,38.4,0.23005958543262706,34.0,70.80020591,37134.0,86934.0,37.0,83.6,38.0,38.0,0.15813924390917247,0.013571651528745943,1.2578363988773091,1.2690477638208295,0.41718076667356846,0.20632844146133847,1655.0,4.8,1.2193158027929234,0.023005958543262706
32.0,17.0,2.294592562651576,7.0,0.14319075822214394,18.0,6.0,103.2,0.12006051049729063,12.0,93.03132567,48794.0,124937.0,33.0,240.2,8.0,9.0,0.1581392439389452,0.013571651536374333,1.257836399145169,1.2690477640730926,0.41718076670642007,0.20632844145449308,1604.0,2.8,0.6803428928179802,0.008004034033152708
33.0,16.0,2.294592562280499,3.0,0.1431907582204469,19.0,17.0,213.6,0.5089537262166648,11.0,100.3088715,52611.0,414576.0,27.0,507.9,14.0,21.0,0.15813924397456677,0.013571651533132646,1.2578363993574158,1.2690477637393385,0.4171807666145652,0.20632844144378837,5579.0,2.8,2.814924163482691,0.142314075103238


In [31]:
df.count()

Out[156]: 2890

In [32]:
df_temp.count()

Out[157]: 2890

In [33]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor, GeneralizedLinearRegression, AFTSurvivalRegression
from pyspark.ml.feature import VectorIndexer

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import DoubleType
 
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder  

In [34]:
valuableColumns = list(df.columns)

In [35]:
valuableColumns

Out[160]: ['Adult_obesity_percentage',
 'Adult_smoking_percentage',
 'All_Specialties_AAMC',
 'Clinical_Care_rank',
 'Emergency_Medicine_specialists_2019',
 'Excessive_drinking_percentage',
 'Health_Behaviors_rank',
 'House_per_sq_mile',
 'ICU_Beds',
 'Length_of_Life_rank',
 'Med_HH_Income_Percent_of_State_Total_2018',
 'Median_Household_Income_2018',
 'POP_ESTIMATE_2018',
 'Physical_Environment_rank',
 'Population_per_sq_mile',
 'Quality_of_Life_rank',
 'Social_Economic_Factors_rank',
 'Surgery_specialists_2019',
 'Total_Hospitals_2019',
 'Total_Primary_Care_Physicians_2019',
 'Total_Specialist_Physicians_2019',
 'Total_nurse_practitioners_2019',
 'Total_physician_assistants_2019',
 'Unemployed_2019',
 'Unemployment_rate_2019',
 'cases',
 'label']

In [36]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [37]:
df=df.rdd.map(lambda x:(Vectors.dense(x[0:-1]), x[-1])).toDF(["features", "label"])
df.show()

+--------------------+--------------------+
 features| label|
+--------------------+--------------------+
[40.0,21.0,2.2945...| 0.5833625014584063|
[32.0,16.0,2.2945...| 0.08473657094823613|
[44.0,20.0,2.2945...| 0.12887013112535842|
[39.0,21.0,2.2945...| 0.0|
[34.0,17.0,2.2945...| 0.10626992561105207|
[36.0,18.0,2.2945...| 0.02596053997923157|
[32.0,17.0,2.2945...| 0.0|
[41.0,17.0,2.2945...|0.023005958543262706|
[32.0,17.0,2.2945...|0.008004034033152708|
[33.0,16.0,2.2945...| 0.142314075103238|
[38.0,19.0,2.2945...| 0.0|
[32.0,17.0,2.2945...| 0.04141604648791926|
[38.0,18.0,2.2945...| 0.1220519753642782|
[39.0,19.0,2.2945...| 0.0|
[42.0,19.0,2.2945...| 0.2940599882376005|
[32.0,15.0,2.2945...|0.018240319862249103|
[37.0,12.0,2.4041...| 0.0|
[36.0,16.0,2.4041...| 0.10591537361648043|
[40.0,15.0,2.4041...| 0.0|
[28.0,13.0,2.4041...| 0.0|
+--------------------+--------------------+
only showing top 20 rows

In [38]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(df)

In [39]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df.randomSplit([0.7, 0.3])

In [40]:
# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

In [41]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

In [42]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)


In [43]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+--------------------+-------------------+--------------------+
 prediction| label| features|
+--------------------+-------------------+--------------------+
 0.03517980748526518|0.03944928794035268|[22.0,13.0,2.2483...|
 0.02248289040043247| 0.0|[22.0,14.0,2.2483...|
 0.3305291327237504| 0.2724300762804214|[22.0,23.0,2.5307...|
0.013624131096633274| 0.0|[23.0,12.0,2.2483...|
 0.06880553481503585| 0.0|[23.0,13.0,2.2483...|
+--------------------+-------------------+--------------------+
only showing top 5 rows

In [44]:
display(predictions)

features,label,indexedFeatures,prediction
"List(1, 26, List(), List(22.0, 13.0, 2.2483920458400726, 55.0, 0.11354670752297921, 18.0, 9.0, 14.6, 0.0, 36.0, 91.99102784, 55776.0, 25349.0, 164.0, 25.8, 14.0, 49.0, 0.12114203806856286, 0.018221825121306563, 1.069757013688903, 1.1810390587399897, 0.34492556140281666, 0.20357576314647519, 326.0, 2.6, 1.0256814864491697))",0.03944928794035268,"List(1, 26, List(), List(22.0, 13.0, 2.2483920458400726, 55.0, 0.11354670752297921, 18.0, 9.0, 14.6, 0.0, 36.0, 91.99102784, 55776.0, 25349.0, 164.0, 25.8, 14.0, 49.0, 0.12114203806856286, 0.018221825121306563, 1.069757013688903, 1.1810390587399897, 0.34492556140281666, 0.20357576314647519, 326.0, 2.6, 1.0256814864491697))",0.03517980748526518
"List(1, 26, List(), List(22.0, 14.0, 2.2483920463594997, 163.0, 0.113546707480091, 17.0, 27.0, 4.8, 0.0, 163.0, 73.92301095, 44821.0, 7032.0, 166.0, 9.6, 158.0, 158.0, 0.12114203811149032, 0.01822182508532423, 1.0697570139362913, 1.181039058589306, 0.3449255614334471, 0.20357576322525597, 118.0, 3.7, 0.42662116040955633))",0.0,"List(1, 26, List(), List(22.0, 14.0, 2.2483920463594997, 163.0, 0.113546707480091, 17.0, 27.0, 4.8, 0.0, 163.0, 73.92301095, 44821.0, 7032.0, 166.0, 9.6, 158.0, 158.0, 0.12114203811149032, 0.01822182508532423, 1.0697570139362913, 1.181039058589306, 0.3449255614334471, 0.20357576322525597, 118.0, 3.7, 0.42662116040955633))",0.02248289040043247
"List(1, 26, List(), List(22.0, 23.0, 2.530720043588812, 90.0, 0.12289494406102433, 17.0, 34.0, 29.5, 0.0, 53.0, 94.10686323, 49280.0, 11012.0, 21.0, 68.9, 35.0, 25.0, 0.17326414584090083, 0.016986680984380675, 1.311076350345078, 1.4923168507083182, 0.8490386276788958, 0.17607064093715946, 168.0, 3.1, 125.40864511442064))",0.2724300762804214,"List(1, 26, List(), List(22.0, 23.0, 2.530720043588812, 90.0, 0.12289494406102433, 17.0, 34.0, 29.5, 0.0, 53.0, 94.10686323, 49280.0, 11012.0, 21.0, 68.9, 35.0, 25.0, 0.17326414584090083, 0.016986680984380675, 1.311076350345078, 1.4923168507083182, 0.8490386276788958, 0.17607064093715946, 168.0, 3.1, 125.40864511442064))",0.3305291327237504
"List(1, 26, List(), List(23.0, 12.0, 2.2483920466247453, 1.0, 0.11354670754365592, 20.0, 2.0, 21.2, 0.0, 11.0, 149.9818578, 90937.0, 45641.0, 167.0, 50.4, 8.0, 7.0, 0.12114203807979669, 0.018221825113384892, 1.0697570138691088, 1.1810390587410444, 0.34492556144694464, 0.20357576315155235, 599.0, 2.7, 0.4382024933721873))",0.0,"List(1, 26, List(), List(23.0, 12.0, 2.2483920466247453, 1.0, 0.11354670754365592, 20.0, 2.0, 21.2, 0.0, 11.0, 149.9818578, 90937.0, 45641.0, 167.0, 50.4, 8.0, 7.0, 0.12114203807979669, 0.018221825113384892, 1.0697570138691088, 1.1810390587410444, 0.34492556144694464, 0.20357576315155235, 599.0, 2.7, 0.4382024933721873))",0.013624131096633274
"List(1, 26, List(), List(23.0, 13.0, 2.248392045404208, 46.808, 0.11354670764119602, 21.0, 48.40382608695652, 0.5, 0.0, 46.90747826086957, 111.2778731, 67470.0, 903.0, 49.479652173913045, 1.0, 47.57217391304348, 47.648, 0.12114203765227022, 0.018221825027685493, 1.0697570143964563, 1.1810390586932447, 0.34492556146179404, 0.2035757630121816, 13.0, 3.1, 2.2148394241417497))",0.0,"List(1, 26, List(), List(23.0, 13.0, 2.248392045404208, 46.808, 0.11354670764119602, 21.0, 48.40382608695652, 0.5, 0.0, 46.90747826086957, 111.2778731, 67470.0, 903.0, 49.479652173913045, 1.0, 47.57217391304348, 47.648, 0.12114203765227022, 0.018221825027685493, 1.0697570143964563, 1.1810390586932447, 0.34492556146179404, 0.2035757630121816, 13.0, 3.1, 2.2148394241417497))",0.06880553481503585
"List(1, 26, List(), List(23.0, 14.0, 2.2483920462146547, 203.0, 0.11354670750988143, 22.0, 63.0, 2.5, 0.0, 10.0, 76.67568281, 46490.0, 6578.0, 14.0, 7.2, 120.0, 155.0, 0.12114203815749469, 0.018221825174825172, 1.069757013833992, 1.18103905868045, 0.34492556141684405, 0.2035757631498936, 59.0, 2.7, 0.6080875646093037))",0.0,"List(1, 26, List(), List(23.0, 14.0, 2.2483920462146547, 203.0, 0.11354670750988143, 22.0, 63.0, 2.5, 0.0, 10.0, 76.6

In [45]:
predictions.dtypes

Out[170]: [('features', 'vector'),
 ('label', 'double'),
 ('indexedFeatures', 'vector'),
 ('prediction', 'double')]

In [46]:


pandas_df_temp = df_temp.toPandas()
pandas_predictions = predictions.toPandas()


/databricks/spark/python/pyspark/sql/dataframe.py:2165: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

In [47]:
pandas_predictions['prediction_de_nom'] = (pandas_predictions['prediction']*pandas_df_temp['POP_ESTIMATE_2018'])/1000


In [48]:
pandas_predictions['label_de_nom'] = (pandas_predictions['label']*pandas_df_temp['POP_ESTIMATE_2018'])/1000

In [49]:
pandas_predictions['prediction_de_nom'] = pandas_predictions['prediction_de_nom'].apply(lambda x: round(x))

In [50]:
pandas_predictions['label_de_nom'] = pandas_predictions['label_de_nom'].apply(lambda x: round(x))

In [51]:
display(pandas_predictions)

features,label,indexedFeatures,prediction,prediction_de_nom,label_de_nom
"List(1, 26, List(), List(22.0, 12.0, 2.2483920462417006, 3.0, 0.11354670752909125, 25.0, 18.0, 445.6, 0.23543675520103016, 5.0, 125.7669218, 76255.0, 1248743.0, 215.0, 1034.4, 9.0, 30.0, 0.12114203811352696, 0.01822182511533598, 1.069757014053332, 1.1810390584772046, 0.34492556138452823, 0.20357576314742104, 19047.0, 2.6, 1.8322425030610783))",0.05765798086555841,"List(1, 26, List(), List(22.0, 12.0, 2.2483920462417006, 3.0, 0.11354670752909125, 25.0, 18.0, 445.6, 0.23543675520103016, 5.0, 125.7669218, 76255.0, 1248743.0, 215.0, 1034.4, 9.0, 30.0, 0.12114203811352696, 0.01822182511533598, 1.069757014053332, 1.1810390584772046, 0.34492556138452823, 0.20357576314742104, 19047.0, 2.6, 1.8322425030610783))",0.07943204200742773,1,1
"List(1, 26, List(), List(22.0, 13.0, 2.2483920458400726, 55.0, 0.11354670752297921, 18.0, 9.0, 14.6, 0.0, 36.0, 91.99102784, 55776.0, 25349.0, 164.0, 25.8, 14.0, 49.0, 0.12114203806856286, 0.018221825121306563, 1.069757013688903, 1.1810390587399897, 0.34492556140281666, 0.20357576314647519, 326.0, 2.6, 1.0256814864491697))",0.03944928794035268,"List(1, 26, List(), List(22.0, 13.0, 2.2483920458400726, 55.0, 0.11354670752297921, 18.0, 9.0, 14.6, 0.0, 36.0, 91.99102784, 55776.0, 25349.0, 164.0, 25.8, 14.0, 49.0, 0.12114203806856286, 0.018221825121306563, 1.069757013688903, 1.1810390587399897, 0.34492556140281666, 0.20357576314647519, 326.0, 2.6, 1.0256814864491697))",0.02206164409514278,7,12
"List(1, 26, List(), List(22.0, 23.0, 2.530720043588812, 90.0, 0.12289494406102433, 17.0, 34.0, 29.5, 0.0, 53.0, 94.10686323, 49280.0, 11012.0, 21.0, 68.9, 35.0, 25.0, 0.17326414584090083, 0.016986680984380675, 1.311076350345078, 1.4923168507083182, 0.8490386276788958, 0.17607064093715946, 168.0, 3.1, 125.40864511442064))",0.2724300762804214,"List(1, 26, List(), List(22.0, 23.0, 2.530720043588812, 90.0, 0.12289494406102433, 17.0, 34.0, 29.5, 0.0, 53.0, 94.10686323, 49280.0, 11012.0, 21.0, 68.9, 35.0, 25.0, 0.17326414584090083, 0.016986680984380675, 1.311076350345078, 1.4923168507083182, 0.8490386276788958, 0.17607064093715946, 168.0, 3.1, 125.40864511442064))",0.18722493037216195,6,8
"List(1, 26, List(), List(23.0, 13.0, 2.248392045404208, 46.808, 0.11354670764119602, 21.0, 48.40382608695652, 0.5, 0.0, 46.90747826086957, 111.2778731, 67470.0, 903.0, 49.479652173913045, 1.0, 47.57217391304348, 47.648, 0.12114203765227022, 0.018221825027685493, 1.0697570143964563, 1.1810390586932447, 0.34492556146179404, 0.2035757630121816, 13.0, 3.1, 2.2148394241417497))",0.0,"List(1, 26, List(), List(23.0, 13.0, 2.248392045404208, 46.808, 0.11354670764119602, 21.0, 48.40382608695652, 0.5, 0.0, 46.90747826086957, 111.2778731, 67470.0, 903.0, 49.479652173913045, 1.0, 47.57217391304348, 47.648, 0.12114203765227022, 0.018221825027685493, 1.0697570143964563, 1.1810390586932447, 0.34492556146179404, 0.2035757630121816, 13.0, 3.1, 2.2148394241417497))",0.07943204200742773,2,0
"List(1, 26, List(), List(24.0, 15.0, 2.24839204610951, 46.808, 0.1135467074927954, 19.0, 48.40382608695652, 0.6, 0.0, 46.90747826086957, 137.4356775, 83330.0, 1388.0, 49.479652173913045, 1.4, 47.57217391304348, 47.648, 0.12114203818443804, 0.01822182492795389, 1.0697570136887609, 1.1810390583573487, 0.3449255612391931, 0.2035757629682997, 17.0, 2.1, 0.7204610951008645))",0.0,"List(1, 26, List(), List(24.0, 15.0, 2.24839204610951, 46.808, 0.1135467074927954, 19.0, 48.40382608695652, 0.6, 0.0, 46.90747826086957, 137.4356775, 83330.0, 1388.0, 49.479652173913045, 1.4, 47.57217391304348, 47.648, 0.12114203818443804, 0.01822182492795389, 1.0697570136887609, 1.1810390583573487, 0.3449255612391931, 0.2035757629682997, 17.0, 2.1, 0.7204610951008645))",0.014671194416998806,0,0
"List(1, 26, List(), List(24.0, 16.0, 2.2483920463585245, 101.0, 0.11354670758469346, 21.0, 162.0, 4.1, 0.0, 161.0, 69.75029687, 42291.0, 7291.0, 109.0, 10.1, 191.0, 86.0, 0.12114203812920038, 0.01822182512686874, 1.069757013852695, 1

In [52]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.186744

In [53]:
treeModel = model.stages[1]
# summary only
print(treeModel)

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_8a2bcf2aef13) of depth 5 with 55 nodes

In [54]:
display(treeModel)

treeNode
"{""index"":31,""featureType"":""continuous"",""prediction"":null,""threshold"":3.6731485281661587,""categories"":null,""feature"":25,""overflow"":false}"
"{""index"":15,""featureType"":""continuous"",""prediction"":null,""threshold"":1.6828083397222535,""categories"":null,""feature"":25,""overflow"":false}"
"{""index"":7,""featureType"":""continuous"",""prediction"":null,""threshold"":0.755370073829801,""categories"":null,""feature"":25,""overflow"":false}"
"{""index"":3,""featureType"":""continuous"",""prediction"":null,""threshold"":0.3247734779552181,""categories"":null,""feature"":25,""overflow"":false}"
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":16827.5,""categories"":null,""feature"":12,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":3.9507327819163896E-4,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":0.007196655390920314,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":45539.0,""categories"":null,""feature"":11,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":0.02248289040043247,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":0.013624131096633274,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [55]:
#print the nodes of the decision tree model
print(treeModel.toDebugString)

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_8a2bcf2aef13) of depth 5 with 55 nodes
 If (feature 25 <= 3.6731485281661587)
 If (feature 25 <= 1.6828083397222535)
 If (feature 25 <= 0.755370073829801)
 If (feature 25 <= 0.3247734779552181)
 If (feature 12 <= 16827.5)
 Predict: 3.9507327819163896E-4
 Else (feature 12 > 16827.5)
 Predict: 0.007196655390920314
 Else (feature 25 > 0.3247734779552181)
 If (feature 11 <= 45539.0)
 Predict: 0.02248289040043247
 Else (feature 11 > 45539.0)
 Predict: 0.013624131096633274
 Else (feature 25 > 0.755370073829801)
 If (feature 18 <= 0.03390573946060715)
 If (feature 25 <= 1.3038126064969062)
 Predict: 0.03517980748526518
 Else (feature 25 > 1.3038126064969062)
 Predict: 0.06263397481021286
 Else (feature 18 > 0.03390573946060715)
 If (feature 16 <= 4.5)
 Predict: 0.055923370327979394
 Else (feature 16 > 4.5)
 Predict: 0.007637035703509906
 Else (feature 25 > 1.6828083397222535)
 If (feature 25 <= 2.756510059770675)
 If (feature 4 <= 0.1682411376110155)
 If (feature 3 <= 23.5)
 Predict: 0.10481312922842098
 Else (feature 3 > 23.5)
 Predict: 0.06880553481503585
 Else (feature 4 > 0.1682411376110155)
 If (feature 12 <= 20283.5)
 Predict: 0.011080823526804512
 Else (feature 12 > 20283.5)
 Predict: 0.152193139794296
 Else (feature 25 > 2.756510059770675)
 If (feature 0 <= 22.5)
 If (feature 23 <= 247.5)
 Predict: 0.7843982258220661
 Else (feature 23 > 247.5)
 Predict: 0.18950134945786362
 Else (feature 0 > 22.5)
 If (feature 1 <= 16.5)
 Predict: 0.11002368472842959
 Else (feature 1 > 16.5)
 Predict: 0.1792061659836148
 Else (feature 25 > 3.6731485281661587)
 If (feature 25 <= 7.302057268537564)
 If (feature 18 <= 0.05131059097087527)
 If (feature 17 <= 0.1740351563702257)
 If (feature 14 <= 13.25)
 Predict: 0.055189170395928
 Else (feature 14 > 13.25)
 Predict: 0.2586469359289981
 Else (feature 17 > 0.1740351563702257)
 If (feature 0 <= 42.5)
 Predict: 0.35297743906853124
 Else (feature 0 > 42.5)
 Predict: 1.1977898611438498
 Else (feature 18 > 0.05131059097087527)
 Predict: 1.2363486503193926
 Else (feature 25 > 7.302057268537564)
 If (feature 18 <= 0.014405884825579068)
 If (feature 5 <= 11.5)
 Predict: 3.0733206497877945
 Else (feature 5 > 11.5)
 If (feature 17 <= 0.13926550517775277)
 Predict: 0.3414066678662533
 Else (feature 17 > 0.13926550517775277)
 Predict: 0.8977850294212883
 Else (feature 18 > 0.014405884825579068)
 If (feature 19 <= 1.401145732780999)
 If (feature 4 <= 0.12589861813688963)
 Predict: 0.3305291327237504
 Else (feature 4 > 0.12589861813688963)
 Predict: 0.625923578647613
 Else (feature 19 > 1.401145732780999)
 If (feature 19 <= 1.5661592691727098)
 Predict: 0.14548395917537463
 Else (feature 19 > 1.5661592691727098)
 Predict: 0.39208603658704194

In [56]:
def extract_feature_imp(feature_imp, dataset, features_col): 
  """Affiche pour chaque features d'un modèle son nom correspondant et les trie dans l'ordre d'importance""" 
  list_extract = [] 
  for i in dataset.schema[features_col].metadata["ml_attr"]["attrs"]: 
    list_extract = list_extract + dataset.schema[features_col].metadata["ml_attr"]["attrs"][i] 
    varlist = pd.DataFrame(list_extract) 
    varlist['score'] = varlist['idx'].apply(lambda x: feature_imp[x]) * 100 
    return varlist.sort_values('score', ascending=False)

In [57]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [58]:
grid = ParamGridBuilder() \
  .addGrid(dt.maxDepth, [2, 3, 4, 5, 6, 7, 8]) \
  .addGrid(dt.maxBins, [2, 4, 8]) \
  .build()

In [59]:
cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=grid, numFolds=3)

In [60]:
# Explicitly create a new run.
# This allows this cell to be run multiple times.
# If you omit mlflow.start_run(), then this cell could run once,
# but a second run would hit conflicts when attempting to overwrite the first run.
import mlflow
import mlflow
import mlflow.mleap
import pyspark
#import pyspark.ml.mleap.SparkUtil 
#import mlflow.mleap.SparkUtil 
import mlflow.mleap
with mlflow.start_run():
  cvModel = cv.fit(trainingData)
  mlflow.set_tag('owner_team', 'UX Data Science') # Logs user-defined tags
  test_metric = evaluator.evaluate(cvModel.transform(testData))
  mlflow.log_metric('testData_' + evaluator.getMetricName(), test_metric) # Logs additional metrics
  mlflow.mleap.log_model(spark_model=cvModel.bestModel, sample_input=testData, artifact_path='dbfs:/databricks/mlflow/2835302286394144') # Logs the best model via mleap


MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.